This is still a **WIP**


# Before we start

In this competition, I have discovered some new signal processing transformations (CQT mainly)
and have explored again ones I konw already (CWT, STFT, and so on). 

So I have decided to take this opportunity as a refresher and share what I have learned.

Notice that most of the implementations will be using PyTorch.

Let's go!

# Fourier Transform

Let's start with the king of transformations: the [Fourier Transform](https://en.wikipedia.org/wiki/Fourier_transform). 

Let's suppose we have a temporal signal $x(t)$, i.e. a sequence of values that change over time. 


For that, let's use one of the competition's signal. Notice that we get 
three signals (one from each observatory) for the price of one file. Neat!

In [ ]:
import numpy as np
import matplotlib.pylab as plt


# Any file will do here:
x = np.load("../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy")



fig, axes = plt.subplots(x.shape[0], 1, figsize=(12, 8))
for i, ax in enumerate(axes):
    ax.plot(x[i, :])
fig.suptitle(f"Time series")


As you can see, some patterns seem to be hidden within the signal. So the next question is: how can we extract these efficiently?

The first "clever" thing that has been done is to try to decompose the temporal signal
into a sum of functions that varies with frequencies. 

To find these functions, we will use the Fourier transform. 

Let's see how it is computed and implemented in practice now.

# Fourier transform: some theory

Here is how it is defined for a signal $x(t)$: 
    
    
$\hat{f}(\xi) = \int_{-\infty}^{\infty} x(t)\ e^{-2\pi i t \xi}\,dt$


the resulting function is a complex valued one in the frequency domain.

For the code part, we will use the PyTorch [FFT](https://pytorch.org/docs/stable/fft.html) module.

In [ ]:
import torch
from torch.fft import fft

# The FFT results are complex numbers.
fft_x = fft(torch.from_numpy(x[0]))

# Plot the module. 
 
fig = plt.figure(figsize=(12, 8))
plt.plot(np.abs(fft_x).reshape(-1))

# From Fourier transform to DFT and FFT

Next step, we need to move from Fourier transform to discrete Fourier transform (or DFT for short).

# What about STFT?

STFT stands for short-term Fourier transform. It is computed by taking moving
windows.



Here is the forumula:  $ and here is how to compute it in more details: 

1. Take 
2. Compute the DFT
    
    
For the implementation, we will be using `librosa.stft`

# Spectrogram: time vs frequency

From the STFT, we get a spectrogram: $ \mathrm{spectrogram}(t,\omega)=\left|\mathrm{STFT}(t,\omega)\right|^2 $

For that we will use some code samples from
this script: https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html

Alright, now that we know how to get frequencies, it is time to plot them vs timestamps.

Spectrogram: https://en.wikipedia.org/wiki/Spectrogram

We can use nnaudio, librosa, or even PyTorch. Here is the code: 


https://pytorch.org/docs/stable/generated/torch.stft.html

https://librosa.org/doc/latest/generated/librosa.stft.html

https://kinwaicheuk.github.io/nnAudio/v0.1.5/_autosummary/nnAudio.Spectrogram.STFT.html

In [ ]:
import librosa
from librosa.display import specshow
import numpy as np


stft_x = librosa.stft(x[0, :])
spectrogram_x = np.abs(stft_x) ** 2


fig, ax = plt.subplots(figsize=(15, 3))




spectrogram_x = librosa.amplitude_to_db(spectrogram_x, ref=np.max)

im = specshow(spectrogram_x, y_axis='log', x_axis='time', ax=ax)

fig.colorbar(im, format='%+2.0f dB')



Not too  much to see. So we need better representations.

# CQT

Here is how it is defined and computed in few steps:

1. We start by computing the short-term Fourier transform (STFT in short): this is a Fourier transform with a moving window over the signal. As a window function, we often use the Hann or Hamming windows. Finally, notice that we will use the discrete version of the STFT. The equation should be:

$X[k,m] = \sum_{n=0}^{N-1} W[n-m] x[n] e^{-j 2 \pi k n/N}.$

2. We introduce the quality factor Q using the following formula: $Q = \frac{f_k}{\delta f_k}.$

3. Now, the window length for the k-th bin is no longer fixed (N) by will vary depending on the filter (we use the previous definition to get a simplified expression):

$N[k] = \frac{f_\text{s}}{\delta f_k} = \frac{f_\text{s}}{f_k} Q.$

4. The digital frequency $\frac{2 \pi k}{N}$ is also changed and the window function now depends on $k$ as well (via the window length $N[k]$)

5. Finaly, replacing the different elements in the original formula, we get:

$X[k] = \frac{1}{N[k]} \sum_{n=0}^{N[k]-1} W[k,n] x[n] e^{\frac{-j2 \pi Qn}{N[k]}}. $

Now, let's see how it can be coded. 

In [ ]:
!pip install nnAudio

In [ ]:
import torch
from nnAudio.Spectrogram import CQT1992v2
import matplotlib.pylab as plt
import numpy as np

# sr is the sampling rate, it is 2048 Hz
# fmax is half the sampling rate

cqt_transform = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)

def run_cqt_transform(x: np.array) -> torch.Tensor:
    # We stack the passed x since there are 3
    # time series per file.
    x = np.hstack(x)
    # Normalize (is there a better way?)
    x = x / np.max(x)
    x = torch.from_numpy(x).float()
    return cqt_transform(x)


img = run_cqt_transform(x)[0]

fig, ax = plt.subplots(figsize=(12, 8)) 
ax.imshow(img)
ax.set_title(f"CQT")

Looks much better: here at least we can see something!
    
Can we do better? Let's try another transformation and check.

# CWT

The next transformation won't strictly speaking based on Fourier transform but
on similar ideas. It uses [wavelets](https://en.wikipedia.org/wiki/Wavelet) as the basis. 

This is the CWT, short for continuous wavelet transform.

Using the following PyTorch library: https://github.com/tomrunia/PyTorchWavelets
=> hard to install!!!

In [ ]:
!git clone https://github.com/ar4/PyTorchWavelets.git > /dev/null
%cd PyTorchWavelets
!pip install -r requirements.txt > /dev/null
!python setup.py install > /dev/null

Here is how to implement it: 

In [ ]:
# There is a notebook that explains how to do it.
from wavelets_pytorch.transform import WaveletTransform, WaveletTransformTorch# PyTorch version


dt = 0.1         # sampling frequency
dj = 0.125       # scale distribution parameter

# Batch of signals to process
# batch = [batch_size x signal_length]

# Initialize wavelet filter banks (scipy and torch implementation)
wa_scipy = WaveletTransform(dt, dj)
# Doesn't work yet?
wa_torch = WaveletTransformTorch(dt, dj, cuda=True)

# Performing wavelet transform (and compute scalogram)
cwt_torch = wa_torch.cwt(torch.tensor(x[0], device="cuda:0").to(torch.float32))

In [ ]:
cwt_torch

# Scalogram: same thing but for wavelets

# Mel and MFCC

If you want even more details, check the following [notebook](https://www.kaggle.com/yassinealouini/what-is-mfcc) I made few months ago.


It explains in more details how Mel spectrogram and MFCCs are computed and how to use them. 

Notice that, in short, they are made from STFT as well and the big difference is that
it is made from a spectrum of a spectrum (a [cepstrum](https://en.wikipedia.org/wiki/Cepstrum)).

# Whitening

One additional useful method is whitening of the signal. To do so.

In [ ]:
# Use the following: from scipy.cluster.vq import whiten

# What about PSD?

## Definition

[PSD](https://en.wikipedia.org/wiki/Spectral_density) is short for power spectral density.

## PSD implementation

Here are scipy methods: https://scipy-lectures.org/intro/scipy/auto_examples/plot_spectrogram.html#generate-a-chirp-signal


https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html#scipy.signal.welch

Let's see what this gives to the original signal.




In [ ]:
from scipy.signal import welch

In [ ]:
fs = 10e5

f, Pxx_den = welch(x[0, :], fs, nperseg=1024)

plt.semilogy(f, Pxx_den)


plt.xlabel('frequency (Hz)')

plt.ylabel('PSD ($V^{2}/Hz$)')

plt.show()

# How to do this with PyTorch?

Here is how to implement PSD using PyTorch.